<a href="https://colab.research.google.com/github/CharKuehl/DermalVOCs/blob/main/PTR_Spectra_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro
For the PTR-MS data we first want to analyse and Plot the data and visualise  peakshifts ect.

#Data 

In [24]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
import torch
import seaborn as sns
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from matplotlib.colors import ListedColormap

AttributeError: ignored

## Data collection from GitHub

####**Tag 1 PTR MS** 
links: BHA \\
rechts: control

In [ ]:
url_0min_left_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.07-12h48m11s_AS.csv"
url_0min_right_control = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.07-13h05m22s_AS.csv"
url_5min_left_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.07-13h31m08s_AS.csv"
url_5min_right_control = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.07-13h50m02s_AS.csv"
url_60min_left_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.07-14h28m11s_AS.csv"
url_60min_right_control = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.07-14h46m02s_AS.csv"
url_120min_left_BHA ="https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.07-15h26m18s_AS.csv"
url_120min_right_control = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.07-15h46m44s_AS.csv"

####**Tag 2 PTR MS** 
links: stripping \\
rechts: BHA

In [ ]:
url2_0min_left_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.08-10h05m53s_AS.csv"
url2_0min_right_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.08-10h26m22s_AS.csv"
url2_5min_left_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.08-10h48m16s_AS.csv"
url2_5min_right_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.08-11h11m09s_AS.csv"
url2_60min_left_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.08-11h40m23s_AS.csv"
url2_60min_right_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.08-12h07m15s_AS.csv"
url2_120min_left_stripping ="https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.08-12h42m08s_AS.csv"
url2_120min_right_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.08-13h07m42s_AS.csv"

####**Tag 3 PTR MS** 
links: stripping \\
rechts: control

In [ ]:
url3_0min_left_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.17-09h28m22s_AS.csv"
url3_0min_right_control = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.17-09h39m47s_AS.csv"
url3_5min_left_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.17-09h56m21s_AS.csv"
url3_5min_right_control = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.17-10h07m39s_AS.csv"
url3_60min_left_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.17-10h39m23s_AS.csv"
url3_60min_right_control = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.17-10h51m19s_AS.csv"
url3_120min_left_stripping ="https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.17-11h23m17s_AS.csv"
url3_120min_right_control = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.17-11h35m36s_AS.csv"

####**Tag 4 PTR MS** 
links: BHA \\
rechts: stripping

In [ ]:
url4_0min_left_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.18-09h06m35s_AS.csv"
url4_0min_right_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.18-09h17m39s_AS.csv"
url4_5min_left_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.18-09h29m19s_AS.csv"
url4_5min_right_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.18-09h42m53s_AS.csv"
url4_60min_left_BHA = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.18-10h16m18s_AS.csv"
url4_60min_right_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.18-10h34m44s_AS.csv"
url4_120min_left_BHA ="https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.18-11h13m02s_AS.csv"
url4_120min_right_stripping = "https://raw.githubusercontent.com/CharKuehl/DermalVOCs/main/DataFile_2022.03.18-11h26m23s_AS.csv"

##Read in BHA data

###07.03.2022

 **07.03. 0 min BHA:**

In [ ]:
df = pd.read_csv(url_0min_left_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0307_0min_left_BHA = df[df['m/z'] > 19]  
df0307_0min_left_BHA = df[df['m/z'] < 200]
#df0307_0min_left_BHA = df[df.m/z <= 20]
#df0307_0min_left_BHA = df.drop(df[df['m/z'] <= 20].index, inplace = True)
#df0307_0min_left_BHA

NameError: ignored

**07.03. 5 min BHA:**

In [ ]:
df = pd.read_csv(url_5min_left_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0307_5min_left_BHA = df[df['m/z'] > 19]  
df0307_5min_left_BHA = df[df['m/z'] < 200]
#df_0min_left_BHA = df[df.m/z <= 20]
#df_0min_left_BHA = df.drop(df[df['m/z'] <= 20].index, inplace = True)
#df0307_5min_left_BHA

**07.03. 60 min BHA:**

In [ ]:
df = pd.read_csv(url_60min_left_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0307_60min_left_BHA = df[df['m/z'] > 19]  
df0307_60min_left_BHA = df[df['m/z'] < 200]
#df_0min_left_BHA = df[df.m/z <= 20]
#df_0min_left_BHA = df.drop(df[df['m/z'] <= 20].index, inplace = True)
#df0307_60min_left_BHA

**07.03. 120 min BHA:**

In [ ]:
df = pd.read_csv(url_120min_left_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0307_120min_left_BHA = df[df['m/z'] > 19]  
df0307_120min_left_BHA = df[df['m/z'] < 200]
#df_0min_left_BHA = df[df.m/z <= 20]
#df_0min_left_BHA = df.drop(df[df['m/z'] <= 20].index, inplace = True)
#df0307_120min_left_BHA

###08.03.2022


**08.03. right 0 min BHA:**

In [ ]:
df = pd.read_csv(url2_0min_right_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0308_0min_right_BHA = df[df['m/z'] > 19]  
df0308_0min_right_BHA = df[df['m/z'] < 200]
df0308_0min_right_BHA

**08.03. right 5 min BHA:**

In [ ]:
df = pd.read_csv(url2_5min_right_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0308_5min_right_BHA = df[df['m/z'] > 19]  
df0308_5min_right_BHA = df[df['m/z'] < 200]
df0308_5min_right_BHA

**08.03. right 60 min BHA:**

In [ ]:
df = pd.read_csv(url2_60min_right_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0308_60min_right_BHA = df[df['m/z'] > 19]  
df0308_60min_right_BHA = df[df['m/z'] < 200]
df0308_60min_right_BHA

**08.03. right 120 min BHA:**

In [ ]:
df = pd.read_csv(url2_120min_right_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0308_120min_right_BHA = df[df['m/z'] > 19]  
df0308_120min_right_BHA = df[df['m/z'] < 200]
df0308_120min_right_BHA

###18.03.2022


**18.03. left 0 min BHA:**

In [ ]:
df = pd.read_csv(url4_0min_left_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0318_0min_right_BHA = df[df['m/z'] > 19]  
df0318_0min_right_BHA = df[df['m/z'] < 200]
df0318_0min_right_BHA

**18.03. left 5 min BHA:**

In [ ]:
df = pd.read_csv(url4_5min_left_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0318_5min_right_BHA = df[df['m/z'] > 19]  
df0318_5min_right_BHA = df[df['m/z'] < 200]
df0318_5min_right_BHA

**18.03. left 60 min BHA:**

In [ ]:
df = pd.read_csv(url4_60min_left_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0318_60min_right_BHA = df[df['m/z'] > 19]  
df0318_60min_right_BHA = df[df['m/z'] < 200]
df0318_60min_right_BHA

**18.03. left 120 min BHA:**

In [ ]:
df = pd.read_csv(url4_120min_left_BHA, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0318_120min_right_BHA = df[df['m/z'] > 19]  
df0318_120min_right_BHA = df[df['m/z'] < 200]
df0318_120min_right_BHA

##Read in Stripping data

###08.03.2022

**08.03. left 0 min:**

In [ ]:
df = pd.read_csv(url2_0min_left_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0308_0min_left_stripping = df[df['m/z'] > 19]  
df0308_0min_left_stripping = df[df['m/z'] < 200]
#df0308_0min_left_stripping

**08.03. left 5 min:**

In [ ]:
df = pd.read_csv(url2_5min_left_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0308_5min_left_stripping = df[df['m/z'] > 19]  
df0308_5min_left_stripping = df[df['m/z'] < 200]
df0308_5min_left_stripping

**08.03. left 60 min:**

In [ ]:
df = pd.read_csv(url2_60min_left_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0308_60min_left_stripping = df[df['m/z'] > 19]  
df0308_60min_left_stripping = df[df['m/z'] < 200]
#df0308_60min_left_stripping

**08.03. left 120 min:**

In [ ]:
df = pd.read_csv(url2_120min_left_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0308_120min_left_stripping = df[df['m/z'] > 19]  
df0308_120min_left_stripping = df[df['m/z'] < 200]
#df0308_120min_left_stripping

###17.03.2022

**17.03. left 0 min:**

In [ ]:
df = pd.read_csv(url3_0min_left_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0317_0min_left_stripping = df[df['m/z'] > 19]  
df0317_0min_left_stripping = df[df['m/z'] < 200]
#df0317_0min_left_stripping

**17.03. left 5 min:**

In [ ]:
df = pd.read_csv(url3_5min_left_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0317_5min_left_stripping = df[df['m/z'] > 19]  
df0317_5min_left_stripping = df[df['m/z'] < 200]
#df0317_5min_left_stripping

**17.03. left 60 min:**

In [ ]:
df = pd.read_csv(url3_60min_left_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0317_60min_left_stripping = df[df['m/z'] > 19]  
df0317_60min_left_stripping = df[df['m/z'] < 200]
#df0317_60min_left_stripping

**17.03. left 120 min:**

In [ ]:
df = pd.read_csv(url3_120min_left_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0317_120min_left_stripping = df[df['m/z'] > 19]  
df0317_120min_left_stripping = df[df['m/z'] < 200]
#df0317_120min_left_stripping

###18.03.2022

**18.03. right stripping 0 min:**

In [ ]:
df = pd.read_csv(url4_0min_right_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0318_0min_right_stripping = df[df['m/z'] > 19]  
df0318_0min_right_stripping = df[df['m/z'] < 200]
df0318_0min_right_stripping

**18.03. right stripping 5 min:**

In [ ]:
df = pd.read_csv(url4_5min_right_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0318_5min_right_stripping = df[df['m/z'] > 19]  
df0318_5min_right_stripping = df[df['m/z'] < 200]
df0318_5min_right_stripping

**18.03. right stripping 60 min:**

In [ ]:
df = pd.read_csv(url4_60min_right_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0318_60min_right_stripping = df[df['m/z'] > 19]  
df0318_60min_right_stripping = df[df['m/z'] < 200]
df0318_60min_right_stripping

**18.03. right stripping 120 min:**

In [ ]:
df = pd.read_csv(url4_120min_right_stripping, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0318_120min_right_stripping = df[df['m/z'] > 19]  
df0318_120min_right_stripping = df[df['m/z'] < 200]
df0318_120min_right_stripping

## Read in control data

###07.03.2022

**07.03. 0 min control:**

In [ ]:
df = pd.read_csv(url_0min_right_control, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0307_0min_right_control = df[df['m/z'] > 19]  
df0307_0min_right_control = df[df['m/z'] < 200]
#df0307_0min_right_control

**07.03. 5 min control:**

In [ ]:
df = pd.read_csv(url_5min_right_control, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0307_5min_right_control = df[df['m/z'] > 19]  
df0307_5min_right_control = df[df['m/z'] < 200]
#df0307_5min_right_control

**07.03. 60 min control:**

In [ ]:
df = pd.read_csv(url_60min_right_control, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0307_60min_right_control = df[df['m/z'] > 19]  
df0307_60min_right_control = df[df['m/z'] < 200]
#df0307_60min_right_control

**07.03. 120 min control:**

In [ ]:
df = pd.read_csv(url_120min_right_control, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0307_120min_right_control = df[df['m/z'] > 19]  
df0307_120min_right_control = df[df['m/z'] < 200]
#df0307_120min_right_control

###17.03.2022

**17.03. 0 min control:**

In [ ]:
df = pd.read_csv(url3_0min_right_control, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0317_0min_right_control = df[df['m/z'] > 19]  
df0317_0min_right_control = df[df['m/z'] < 200]
df0317_0min_right_control

**17.03. 5 min control:**

In [ ]:
df = pd.read_csv(url3_5min_right_control, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0317_5min_right_control = df[df['m/z'] > 19]  
df0317_5min_right_control = df[df['m/z'] < 200]
df0317_5min_right_control

**17.03. 60 min control:**

In [ ]:
df = pd.read_csv(url3_60min_right_control, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0317_60min_right_control = df[df['m/z'] > 19]  
df0317_60min_right_control = df[df['m/z'] < 200]
#df0317_60min_right_control

**17.03. 120 min control:**

In [ ]:
df = pd.read_csv(url3_120min_right_control, sep= ";")
df = df.iloc[:,0:]
df = df.drop(columns=["Current_Mass", "Current"])
df = df.rename(columns={"Average_Mass": "m/z", "Average": "intensity"})
df0317_120min_right_control = df[df['m/z'] > 19]  
df0317_120min_right_control = df[df['m/z'] < 200]
#df0317_120min_right_control

#Plotting libraries 1st try 
Here some information about building interactive plots in Python can be found: [Interactive Python Plot Libraries](https://mode.com/blog/python-interactive-plot-libraries/)

###Seaborn

In [ ]:
#VOCs = sns.load_dataset("df_0min_left_BHA")
   
df_0min_left_BHA.plot.line(x = "m/z", y = "intensity")


###Pygal 
[link text](https://)


In [ ]:
pip install pygal

###Bokeh

In [ ]:
pip install bokeh

In [ ]:
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show

output_notebook()


#change theme for visibility
curdoc().theme = 'dark_minimal' #caliber, light_minimal, dark_minimal, night_sky, contrast

# prepare data
x = df0307_0min_left_BHA['m/z']
y = df0307_0min_left_BHA['intensity']

# create a new plot with a title and axis labels
p = figure (title='VOC sample plot comparision 0-120 min', x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)) #, x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)
# add a line renderer with legend and line thickness
p.line(x, y, legend_label="VOCs", line_width=2)

# show the results
#output_file("voc.html")

show(p)

In [ ]:
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show

output_notebook()


#change theme for visibility
curdoc().theme = 'dark_minimal' #caliber, light_minimal, dark_minimal, night_sky, contrast

# prepare data
x0 = df0307_0min_left_BHA['m/z']
y0 = df0307_0min_left_BHA['intensity']
x1 = df0307_5min_left_BHA['m/z']
y1 = df0307_5min_left_BHA['intensity']
x2 = df0307_60min_left_BHA['m/z']
y2 = df0307_60min_left_BHA['intensity']
x3 = df0307_120min_left_BHA['m/z']
y3 = df0307_120min_left_BHA['intensity']

# create a new plot with a title and axis labels
p = figure (title='VOC sample plot comparision 0-120 min', x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)) #, x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)
# add a line renderer with legend and line thickness
#p.line(x, y, legend_label="VOCs", line_width=2)

# add multiple renderers
#p.line(x0, y0, legend_label="0 min", color="blue", line_width=2)
p.line(x1, y1, legend_label="5 min", color="red", line_width=2)
p.line(x2, y2, legend_label="60 min", color="green", line_width=2)
p.line(x3, y3, legend_label="120 min", color="yellow", line_width=2)

# show the results
#output_file("voc.html")

show(p)

In [ ]:
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show

output_notebook()


#change theme for visibility
curdoc().theme = 'dark_minimal' #caliber, light_minimal, dark_minimal, night_sky, contrast

# prepare data
x0 = df0307_0min_right_control['m/z']
y0 = df0307_0min_right_control['intensity']
x1 = df0307_5min_right_control['m/z']
y1 = df0307_5min_right_control['intensity']
x2 = df0307_60min_right_control['m/z']
y2 = df0307_60min_right_control['intensity']
x3 = df0307_120min_right_control['m/z']
y3 = df0307_120min_right_control['intensity']

# create a new plot with a title and axis labels
p = figure (title='VOC sample plot comparision 0-120 min control', x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)) #, x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)
# add a line renderer with legend and line thickness
#p.line(x, y, legend_label="VOCs", line_width=2)

# add multiple renderers
p.line(x0, y0, legend_label="0 min", color="blue", line_width=2)
p.line(x1, y1, legend_label="5 min", color="red", line_width=2)
p.line(x2, y2, legend_label="60 min", color="green", line_width=2)
p.line(x3, y3, legend_label="120 min", color="yellow", line_width=2)

# show the results
#output_file("voc.html")

show(p)

Plot of 0min left arm BHA day 1

#Spectra comparision and analysis PTR

## Changes in spectra over time ompared to control 
In this part I want to subtract the changes between the inital treatment, 60 minutes and 120 minutes later and plot these subtractions compared to those of the control samples. 

### Variance within the control samples
So far I have gathered two control measurements on the 7th and 17th of March. I want to subtract the spectra from one another to visualize the variance to better understand the data. Idealy We would wish for completely straight lines to assure that the variance within the measuring period of controls is negligible. 

In [ ]:
#df0307_0min_right_control
#df0307_5min_right_control
#df0307_60min_right_control
#df0307_120min_right_control
df0307_control_subtract0_5 = df0307_0min_right_control.subtract(df0307_5min_right_control, axis = 1)
df0307_control_subtract0_60 = df0307_0min_right_control.subtract(df0307_60min_right_control, axis = 1)
df0307_control_subtract5_60 = df0307_5min_right_control.subtract(df0307_60min_right_control, axis = 1)

In [ ]:
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show

output_notebook()


#change theme for visibility
curdoc().theme = 'dark_minimal' #caliber, light_minimal, dark_minimal, night_sky, contrast

# prepare data
x = df0307_control_subtract5_60['m/z']
y = df0307_control_subtract5_60['intensity']

# create a new plot with a title and axis labels
p = figure (title='VOC subtraction plot control 0-60min', x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)) #, x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)
# add a line renderer with legend and line thickness
p.line(x, y, legend_label="VOCs", line_width=2)

# show the results
#output_file("voc.html")

show(p)

### Variance within the BHA samples

In [ ]:
#df0307_0min_left_BHA
#df0307_5min_left_BHA
#df0307_60min_left_BHA
#df0307_120min_left_BHA
df0307_BHA_subtract0_5 = df0307_0min_left_BHA.subtract(df0307_5min_left_BHA, axis = 1)
#df0307_control_subtract0_60 = df0307_0min_right_control.subtract(df0307_60min_right_control, axis = 1)
#df0307_control_subtract5_60 = df0307_5min_right_control.subtract(df0307_60min_right_control, axis = 1)
df0307_BHA_subtract0_5

In [ ]:
df0307_BHA_subtract0_5 = df0307_0min_left_BHA.subtract(df0307_5min_left_BHA, axis = 1)

from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show

output_notebook()


#change theme for visibility
curdoc().theme = 'dark_minimal' #caliber, light_minimal, dark_minimal, night_sky, contrast

# prepare data
x = df0307_BHA_subtract0_5['m/z']
y = df0307_BHA_subtract0_5['intensity']

# create a new plot with a title and axis labels
p = figure (title='VOC subtraction plot control 0-60min', x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)) #, x_axis_label='m/z', y_axis_label='intensity [ppb]',width=900, height=400, x_range=(30, 70), y_range=(0, 4000)
# add a line renderer with legend and line thickness
p.line(x, y, legend_label="VOCs", line_width=2)

# show the results
#output_file("voc.html")

show(p)